# Baseline Model

In [38]:
%load_ext autoreload
%autoreload 2

import sys
import infostop
import pyproj
    
sys.path.append('./src')
from data_utils import *
clear_output(wait=False)

### Load Train, validation and test data

In [39]:
user_train, user_val, user_test = train, val, test = train_test_data_split()

In [40]:
data_train = pd.concat([create_data_frame(*load_user_data(user, load_wgs=True), segmentation=True) for user in user_train]).reset_index(drop=True)
data_val = pd.concat([create_data_frame(*load_user_data(user, load_wgs=True), segmentation=True) for user in user_val]).reset_index(drop=True)
data_test = pd.concat([create_data_frame(*load_user_data(user, load_wgs=True), segmentation=True) for user in user_test]).reset_index(drop=True)

Segmentation is not strictly needed for the Baseline model, but in order to ensure the datesets accross all three models are the same we still perform it here, and purge all points that would otherwise been purged to build the sequences.

In [41]:
data_train = data_train[data_train['segment_ix'] >= 5]
data_val = data_val[data_val['segment_ix'] >= 5]
data_test = data_test[data_test['segment_ix'] >= 5]

In [ ]:
labels = infostop.label_trace(data_val[['long', 'lat']].values)

In [53]:
pred = np.zeros_like(labels)
pred[labels >= 0] = 1

In [54]:
print(classification_report(data_val['label'], pred, target_names = ['Motion','Stop']))

              precision    recall  f1-score   support

      Motion       0.73      0.58      0.65     43215
        Stop       0.78      0.87      0.82     72334

   micro avg       0.76      0.76      0.76    115549
   macro avg       0.75      0.73      0.74    115549
weighted avg       0.76      0.76      0.76    115549



In [59]:
labels = infostop.label_trace(data_test[['lat', 'long']].values)

In [60]:
pred = np.zeros_like(labels)
pred[labels >= 0] = 1

In [61]:
print(classification_report(data_test['label'], pred, target_names = ['Motion','Stop']))

              precision    recall  f1-score   support

      Motion       0.74      0.73      0.74     42425
        Stop       0.91      0.91      0.91    122881

   micro avg       0.87      0.87      0.87    165306
   macro avg       0.83      0.82      0.82    165306
weighted avg       0.87      0.87      0.87    165306

